In [2]:
import regionmask
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import json
import glob

from climaf.api import *



pattern = '/data/jservon/IPCC/tx35/bias_corrected/individual_models/CMIP6_${experiment}_${variable}_${period}_${member}.nc'
cproject('tx_individual_models_cmip6_ch12','experiment','period','member',('variable','tx35isimip'), ensemble=['member'], separator='%')
dataloc(project='tx_individual_models_cmip6_ch12', url=pattern)    


exp_dict_list = dict(
    baseline = dict(experiment='historical',
         period='1995-2014'
        ),
    ssp126_mid = dict(experiment='ssp126',
         period='2041-2060'
        ),
    ssp126_far = dict(experiment='ssp126',
         period='2081-2100'
        ),
    ssp585_mid = dict(experiment='ssp585',
         period='2041-2060'
        ),
    ssp585_far = dict(experiment='ssp585',
         period='2081-2100'
        )  
)

# -- Make the ensembles per experiments
ens_exp_dict = dict()
lom_per_exp = dict()
for exp in exp_dict_list:
    
    experiment = exp_dict_list[exp]['experiment']
    period = exp_dict_list[exp]['period']

    req = ds(experiment=experiment,
             period=period,
             member='*',
             project='tx_individual_models_cmip6_ch12')

    ens_exp = req.explore('ensemble')
    #
    if 'EC-Earth3-Veg_r1i1p1f1' in ens_exp:
        ens_exp.pop('EC-Earth3-Veg_r1i1p1f1')

    # -- Climatologies
    clim_exp      = clim_average(ccdo(ens_exp, operator='yearsum'), 'ANM')

    # -- Changes = Scenario minus baselines
    ens_exp_dict[exp] = clim_exp
    lom_per_exp[exp] = clim_exp.keys()

req_dict = dict(project='tx_individual_models_cmip6_ch12')


# -- Get the GWL info
import csv
GWL_csv = '/home/jservon/Chapter12_IPCC/scripts/ATLAS/warming-levels/CMIP6_Atlas_WarmingLevels.csv'

GWL_dict = dict()
i = 0
with open(GWL_csv) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')#, quotechar='|')
    for row in spamreader:
        print row
        model = row[0]#.split('_')[0]
        print model
        GWL_dict[model] = dict()
        if i==0:
            colnames = row
        j = 1
        for elt in row[1:len(row)]:
            print elt
            GWL_dict[model][colnames[j]] = row[j]
            j = j + 1
        i = i + 1
        
# -- Retrieve the GWL in the models
ens_dict_per_GWL = dict()
list_of_GWLs = ['1.5','2','3','4']
for GWL in list_of_GWLs:
    ens_dict_per_GWL[GWL] = dict()


for scenario in ['26','85']:
    
    if scenario=='26': req_scenario = 'ssp126'
    if scenario=='85': req_scenario = 'ssp585'
    list_of_models = ens_exp_dict[req_scenario+'_far'].keys()
    for wmodel_realization in list_of_models:
        wmodel = wmodel_realization #.split('_')[0]
        if wmodel in GWL_dict:
            print 'We have : ', wmodel
            print GWL_dict[wmodel]
            for GWL in list_of_GWLs:
                if scenario=='26': GWL_scenario = GWL+'_ssp126'
                if scenario=='85': GWL_scenario = GWL+'_ssp585'

                # --> file nc
                # --> period
                central_year = GWL_dict[wmodel][GWL_scenario]
                if central_year not in ['NA','9999'] and float(central_year)>=2024:
                    start_year = str( int(central_year)-9 )
                    end_year = str( int(central_year)+10 )
                    
                    dat = ds(member = wmodel_realization,
                             experiment = req_scenario,
                             period=start_year+'-'+end_year,
                             **req_dict
                             )
                    ens_dict_per_GWL[GWL][wmodel+'_'+scenario] = clim_average(ccdo(dat, operator='yearsum'), 'ANM')
                    print cfile(ens_dict_per_GWL[GWL][wmodel+'_'+scenario])
        else:
            print '   ==> We dont have GWL info for ',wmodel,' in GWL_dict = ',GWL_dict.keys()


['', '1.5_ssp126', '2_ssp126', '3_ssp126', '4_ssp126', '1.5_ssp245', '2_ssp245', '3_ssp245', '4_ssp245', '1.5_ssp370', '2_ssp370', '3_ssp370', '4_ssp370', '1.5_ssp585', '2_ssp585', '3_ssp585', '4_ssp585']

1.5_ssp126
2_ssp126
3_ssp126
4_ssp126
1.5_ssp245
2_ssp245
3_ssp245
4_ssp245
1.5_ssp370
2_ssp370
3_ssp370
4_ssp370
1.5_ssp585
2_ssp585
3_ssp585
4_ssp585
['ACCESS-CM2_r1i1p1f1', '2027', '2042', 'NA', 'NA', '2028', '2040', '2070', 'NA', '2027', '2039', '2062', '2082', '2025', '2038', '2055', '2070']
ACCESS-CM2_r1i1p1f1
2027
2042
NA
NA
2028
2040
2070
NA
2027
2039
2062
2082
2025
2038
2055
2070
['ACCESS-ESM1-5_r1i1p1f1', '2030', '2072', 'NA', 'NA', '2029', '2045', 'NA', 'NA', '2033', '2048', '2069', 'NA', '2027', '2038', '2060', '2078']
ACCESS-ESM1-5_r1i1p1f1
2030
2072
NA
NA
2029
2045
NA
NA
2033
2048
2069
NA
2027
2038
2060
2078
['AWI-CM-1-1-MR_r1i1p1f1', '2022', '2050', 'NA', 'NA', '2020', '2039', 'NA', 'NA', '2021', '2037', '2064', 'NA', '2019', '2036', '2059', '2079']
AWI-CM-1-1-MR_r1i1p

/data/jservon/climafcache/66/bcc54f34c88431836d3ffb1afda95f4e05b608b64b08d2ebb0bd09.nc
/data/jservon/climafcache/6f/4eb0b5acf4cfdbfbe28f5837a6bd654a80fee69741b166c3f0ec00.nc
/data/jservon/climafcache/00/cbd693e73b4bfad3c4a1d1b45435c7790254813d3a1d0d20c339e0.nc
We have :  NorESM2-MM_r1i1p1f1
{'2_ssp126': 'NA', '2_ssp585': '2054', '4_ssp126': 'NA', '3_ssp585': '2076', '1.5_ssp245': '2046', '1.5_ssp370': '2046', '1.5_ssp126': 'NA', '1.5_ssp585': '2039', '3_ssp370': 'NA', '4_ssp585': 'NA', '4_ssp370': 'NA', '2_ssp245': '2077', '3_ssp245': 'NA', '2_ssp370': '2062', '3_ssp126': 'NA', '4_ssp245': 'NA'}
/data/jservon/climafcache/8e/d4fc0ddf3696b0e3aab3a82d6efb91970824be3ebc51b149bec2d2.nc
/data/jservon/climafcache/d8/4c00db212584198855b77bb686c12cce746a1f8ce91f7b1374abd4.nc
/data/jservon/climafcache/5c/09a969ad6a8b97b57523bd3fcb7773d22ac99ef3bbd43b56b6fe9c.nc
We have :  NorESM2-LM_r1i1p1f1
{'2_ssp126': 'NA', '2_ssp585': '2056', '4_ssp126': 'NA', '3_ssp585': '2077', '1.5_ssp245': '2055', '1.5_s

In [ ]:
  ==> We dont have GWL info for  MPI-ESM1-2-HR_r1i1p1f1  in GWL_dict =  ['', 'A